# Evidencia Final: Entornos Multiagentes

**Integrantes:**
- Karla González Sánchez A01541526
- Luis Ángel Alba Alfaro A01640314
- María Fernanda Elizalde Macías A01634135
- Sofía del Pilar Batiz Martínez A01634125

**Modelación de sistemas multiagentes con gráficas computacionales** 
- Gamaliel Abisay Palomo Briones  
- Omar Mendoza Montoya  
- Guillermo Gabriel Rivas Aguilar 
   
30 de noviembre de 2022  

### Importar librerías

In [ ]:
import agentpy as ap 
import numpy as np 
import matplotlib.pyplot as plt
import math
import IPython
import random
import json

### Definir las clases de agentes

#### Clase de Semáforos

In [ ]:
class TrafficLight(ap.Agent):
    def setup(self):
        self.step_time = 0.1
        self.direction = [0, 1] #0 -> sur, 1 -> norte
        self.color = 'green'

        self.color_time = 0
        self.green_time = 0
        self.yellow_time = 0
        self.red_time = 0

    def set_green(self):
        self.color = 'green'
        self.color_time = 0
        
    def set_yellow(self):
        self.color = 'yellow'
        self.color_time = 0
        
    def set_red(self):
        self.color = 'red'
        self.color_time = 0
        
    def update(self):
        self.color_time += self.step_time

        if self.color == 'green' and self.color_time >= self.green_time:
            self.set_yellow()
            
        if self.color == 'yellow' and self.color_time >= self.yellow_time:
            self.set_red()
            
        if self.color == 'red' and self.color_time >= self.red_time:
            self.set_green()


#### Clase de Auto

In [ ]:
class Car(ap.Agent):
    def setup(self):
        self.step_time = 0.1
        self.direction = [0, 1] #0 -> sur, 1 -> norte
        self.speed = 0.0
        self.max_speed = 16.66
        self.state = 0 #0 -> regular, 1 -> choque
    
    def update_position(self):
        if self.state == 1:
            return

        pos = [self.speed * self.step_time * self.direction[0], self.speed * self.step_time * self.direction[1]]
        self.model.avenue.move_by(self, pos)
    
    def update_speed(self):
        if self.state == 1:
            return
        
        pos_init = self.model.avenue.positions[self]
        minimum_car_separation = 1000000000000

        for car in self.model.cars:
            if car == self:
                continue

            pos_final = self.model.avenue.positions[car]
            d1 = pos_final[0] - pos_init[0]
            d2 = pos_final[1] - pos_init[1]

            car1 = self.direction[0] * car.direction[0] + self.direction[1] * car.direction[1]
            car2 = d1 * self.direction[0] + d2 * self.direction[1]

            if car1 > 0 and car2 > 0:
                res = math.sqrt(d1**2 + d2**2) #producto punto
                if res < minimum_car_separation:
                    minimum_car_separation = res
        
        traffic_light_color = 'green'
        minimum_traffic_light_separation = 1000000000000

        for traffic_light in self.model.traffic_lights:
            pos_final = self.model.avenue.positions[traffic_light]
            d1 = pos_final[0] - pos_init[0]
            d2 = pos_final[1] - pos_init[1]

            tl1 = self.direction[0] * traffic_light.direction[0] + self.direction[1] * traffic_light.direction[1]
            tl2 = d1 * self.direction[0] + d2 * self.direction[1]

            if tl1 < 0 and tl2 > 0:
                res = math.sqrt(d1**2 + d2**2)
                if res < minimum_traffic_light_separation:
                    minimum_traffic_light_separation = res
                    traffic_light_color = traffic_light.color
        
        if minimum_car_separation < 2:
            self.speed = 0
            self.state = 1
            
        elif minimum_car_separation < 5:
            self.speed = 0
            self.state = 0
        
        elif minimum_car_separation < 10:
            self.speed = np.maximum(self.speed - 10 * self.step_time, 0)
        
        elif minimum_car_separation < 15:
            self.speed = np.maximum(self.speed - 5 * self.step_time, 0)

        elif minimum_traffic_light_separation < 20 and traffic_light_color == 'yellow':
            self.speed = np.minimum(self.speed + 2.5 * self.step_time, self.max_speed)
            
        elif minimum_traffic_light_separation < 30 and traffic_light_color == 'red':
            self.speed = np.maximum(self.speed - 10 * self.step_time, 0)

        elif minimum_traffic_light_separation < 40 and traffic_light_color == 'yellow':
            self.speed = np.maximum(self.speed - 2.5 * self.step_time, 5)

        elif minimum_traffic_light_separation < 50 and traffic_light_color == 'red':
            self.speed = np.maximum(self.speed - 2.5 * self.step_time, 5)

        else:
            self.speed = np.minimum(self.speed + 2.0 * self.step_time, self.max_speed)
   

### Definir clase del Modelo

In [ ]:
class AvenueModel(ap.Model):
    def setup(self):
        self.crashes = 0
        self.cars = ap.AgentList(self, self.p.cars, Car)
        self.cars.step_time = self.p.step_time
        self.cars.speed = self.p.v0

        cars_north = self.p.cars // 2
        cars_south = self.p.cars - cars_north

        for i in range(cars_north):
            self.cars[i].direction = [0, 1]
        
        for i in range(cars_south):
            self.cars[cars_north + i].direction = [0, -1]
        
        self.traffic_lights = ap.AgentList(self, 2, TrafficLight)
        self.traffic_lights.step_time = self.p.step_time
        self.traffic_lights.green_time += self.p.green_time
        self.traffic_lights.yellow_time += self.p.yellow_time
        self.traffic_lights.red_time += self.p.red_time
        self.traffic_lights[0].direction = [0, 1]
        self.traffic_lights[1].direction = [0, -1]

        self.avenue = ap.Space(self, shape = [60, self.p.size], torus = True)
        self.avenue.add_agents(self.traffic_lights, random = True)
        self.avenue.move_to(self.traffic_lights[0], [10, self.p.size * 0.5 - 5])
        self.avenue.move_to(self.traffic_lights[1], [50, self.p.size * 0.5 + 5])

        self.avenue.add_agents(self.cars, random = True)

        for i in range(cars_north):
            self.avenue.move_to(self.cars[i], [40, 10 * (i + 1)])
        
        for i in range(cars_south):
            self.avenue.move_to(self.cars[i + cars_north], [20, self.p.size - 10 * (i + 1)])

        #Creacion del JSON
        if self.p.getJson == True:
            self.frames = 0
            self.data = {}
            self.data['frames'] = []

    def step(self):
        self.traffic_lights.update()
        self.cars.update_position()
        self.cars.update_speed()

        #Creacion del JSON
        if self.p.getJson == True:
            cars_list = []
            traffic_lights_list = []

            for i in range(self.p.cars):
                cars_list.append({
                    'id': i,
                    'x': self.model.avenue.positions[self.cars[i]][0],
                    'z': self.model.avenue.positions[self.cars[i]][1],
                    'dir': math.atan2(self.cars[i].direction[0], self.cars[i].direction[1]) * 180 / math.pi
                })
            
            for i in range(len(self.traffic_lights)):
                traffic_lights_list.append({
                    'id': i + self.model.p.cars,
                    'color': self.traffic_lights[i].color
                })

            self.data['frames'].append({
                'frame': self.frames,
                'cars': cars_list,
                'traffic_lights': traffic_lights_list
            })

            self.frames += 1

        
    def update(self):
        self.crashes = len(self.cars.select(self.cars.state == 1))
        self.record('Crashes', self.crashes)
       
    def end(self):
        self.crashes = len(self.cars.select(self.cars.state == 1))
        self.report('Crashes', self.crashes)

        #Creacion del JSON
        if self.p.getJson == True:
            file_json = json.dumps(self.data)
            with open('data.json', 'w') as outfile:
                outfile.write(file_json)

#### Definir la animación

In [ ]:
def animation_plot_single(model, ax):
    ax.set_title(f"Simulacion de una Avenida\n"
                 f"Time-step: {model.t * model.p.step_time:.2f}\n"
                 f"Crashes: {model.crashes}")

    tl1_pos = model.avenue.positions[model.traffic_lights[0]]
    ax.scatter(*tl1_pos, s = 20, c = model.traffic_lights[0].color)
    tl2_pos = model.avenue.positions[model.traffic_lights[1]]
    ax.scatter(*tl2_pos, s = 20, c = model.traffic_lights[1].color)

    ax.set_xlim(0, model.avenue.shape[0])
    ax.set_ylim(0, model.avenue.shape[1])

    for car in model.cars:
        car_pos = model.avenue.positions[car]
        ax.scatter(*car_pos, s = 20, c = 'black')
    
    ax.set_axis_off()
    ax.set_aspect('equal', 'box')

def animation_plot(model, params):
    fig = plt.figure(figsize = (10, 10))
    ax = fig.add_subplot(111)
    animation = ap.animate(model(params), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps = 15))

### Simulación

#### Definir parametros

In [ ]:
parameters = {
    'steps' : 1000,
    'step_time' : 0.1,
    'size' : 100,
    'green_time' : random.randint(5,10),
    'yellow_time' : 5,
    'red_time' : 60,
    'cars' : 10,
    'v0' : 0,
    'getJson' : True
}

#### Ejecutar el modelo

In [ ]:
model = AvenueModel(parameters)
results = model.run()

#### Ver los reportes de choques

In [ ]:
results.reporters #En total

#### Ver animación

In [ ]:
animation_plot(AvenueModel, parameters)